In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('TotalAmountbyCostumers').master('local').getOrCreate()

sc = spark.sparkContext

23/10/08 16:03:36 WARN Utils: Your hostname, AHE resolves to a loopback address: 127.0.1.1; using 192.168.121.8 instead (on interface wlo1)
23/10/08 16:03:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/08 16:03:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/08 16:03:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [11]:
def process_line(line):
    """
    Process csv like table and perform basic processing
    """    
    # Separate into columns
    line = line.split(',')

    # Remove index - There is no index
    
    # Set proper dtypes and return
    return (int(line[0]), int(line[1]), float(line[2]))



# Columns: CustomerID, ProductID, Price
# Example: 44,         8602,      37.19'
lines = sc.textFile('customer-orders.csv')

# Basic processing and proper dtypes
rdd = lines.map(process_line)

rdd.take(5)

[(44, 8602, 37.19),
 (35, 5368, 65.89),
 (2, 3391, 40.64),
 (47, 6694, 14.98),
 (29, 680, 13.08)]

In [16]:
# Get purchases by customer
purchasesByCustomer = rdd.mapValues(lambda x: 1).reduceByKey(lambda x, y: x + y)
purchasesByCustomer.take(10)

[(44, 92),
 (35, 110),
 (2, 118),
 (47, 86),
 (29, 110),
 (91, 91),
 (70, 108),
 (85, 105),
 (53, 94),
 (14, 100)]

In [68]:
# Get amount wasted by customer
byCustID = rdd.map(lambda x: (x[0], x[2]))
byCustID.reduceByKey(lambda x, y: x + y).take(5)

[(44, 4756.8899999999985),
 (35, 5155.419999999999),
 (2, 5994.59),
 (47, 4316.299999999999),
 (29, 5032.529999999999)]

In [91]:
# Get mean wasted by purchase for every customer, sorted descending
(
    rdd
    .map(lambda x: (x[0], (x[2], 1)))
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
    .mapValues(lambda x: x[0] / x[1])
    .sortBy(lambda x: x[1], ascending = False)
    .take(5)
)

[(42, 56.96840000000003),
 (77, 55.48371794871793),
 (68, 55.43869565217389),
 (71, 55.006055045871584),
 (80, 54.97511627906977)]